In [1]:
#f'(x[i]) = (f(x[i+1]) - f(x[i]))/ step

import pandas as pd


#discretisation

def euler(beta, gama, mu, S, I, R, D, step, time):
    time_update = time[-1] + step

    S_update = (-beta * S[-1] * I[-1])/ step + S[-1]
    I_update = (beta * S[-1] * I[-1] - gama * I[-1] - mu * I[-1])/ step + I[-1]
    R_update = (gama * I[-1])/ step + R[-1]
    D_update = (mu * I[-1])/ step + D[-1]

    S.append(S_update)
    I.append(I_update)
    R.append(R_update)
    D.append(D_update)
    time.append(time_update)
    dataframe = pd.DataFrame({'Time': time, 'S': S, 'I': I, 'R': R, 'D': D})
    return dataframe

       Time             S             I             R             D
0      0.00  9.900000e+01  1.000000e+00  0.000000e+00  0.000000e+00
1      0.01 -4.851000e+03  4.934500e+03  1.500000e+01  1.500000e+00
2      0.02  1.196858e+09 -1.196939e+09  7.403250e+04  7.403250e+03
3      0.03  7.162834e+19 -7.162834e+19 -1.795402e+10 -1.795402e+09
4      0.04  2.565309e+41 -2.565309e+41 -1.074425e+21 -1.074425e+20
...     ...           ...           ...           ...           ...
9995  99.95           NaN           NaN           NaN           NaN
9996  99.96           NaN           NaN           NaN           NaN
9997  99.97           NaN           NaN           NaN           NaN
9998  99.98           NaN           NaN           NaN           NaN
9999  99.99           NaN           NaN           NaN           NaN

[10000 rows x 5 columns]


In [ ]:
euler(beta, gama, mu, S, I, R, D, step, time)

,Time,S,I,R,D
0,0.00,9.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00
1,0.01,-4.851000e+03,4.934500e+03,1.500000e+01,1.500000e+00
2,0.02,1.196858e+09,-1.196939e+09,7.403250e+04,7.403250e+03
3,0.03,7.162834e+19,-7.162834e+19,-1.795402e+10,-1.795402e+09
4,0.04,2.565309e+41,-2.565309e+41,-1.074425e+21,-1.074425e+20
...,...,...,...,...,...
9997,99.97,NaN,NaN,NaN,NaN
9998,99.98,NaN,NaN,NaN,NaN
9999,99.99,NaN,NaN,NaN,NaN
10000,100.00,NaN,NaN,NaN,NaN
